<a href="https://colab.research.google.com/github/nicolaiberk/bild/blob/main/np_slant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Estimating migration slant in german newspapers using bert

This notebook tries to estimate the sentiment in german newspapers' migration coverage over time. It will draw a subset of articles for each year, cut them into sentences, filter those containing migration-related terms (identified earlier using dictioary expansion with word embeddings), and estimate the sentiment in these sentences using BERT transformer models.

In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 5.1MB/s 
     |████████████████████████████████| 901kB 24.0MB/s 
     |████████████████████████████████| 3.3MB 41.1MB/s 


In [2]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# define input data and migration dictionary
url_list = ["https://www.dropbox.com/s/fg2j14sckivbv9e/_bild_articles.csv?dl=1",
            "https://www.dropbox.com/s/gu74lpyys9g30vk/_faz_articles.csv?dl=1",
            "https://www.dropbox.com/s/qqvd9qbgd16q6ny/_spon_articles.csv?dl=1",
            "https://www.dropbox.com/s/53yeud52h3r1hc4/_sz_articles.csv?dl=1",
            "https://www.dropbox.com/s/2atgbzx4dzbq6nd/_taz_articles.csv?dl=1",
            "https://www.dropbox.com/s/c1gmzcriuh337vd/_weltonline_articles.csv?dl=1"]
paper_list = ["bild", "faz", "spon", "sz", "taz", "weltonline"]
mig_dict = mig_dict = pd.read_csv("https://www.dropbox.com/s/65n22q1l19xkkmu/german_glove.csv?dl=1", encoding="latin-1")["x"]

In [4]:
# setup BERT
model_name = "oliverguhr/german-sentiment-bert"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [ ]:
for url, paper in zip(url_list, paper_list):
  
  # load data
  dta = pd.read_csv(url)
  dta = dta[dta.text == dta.text] # gets rid of missings
  dta = dta.reset_index(drop=True)
  dta["mig_sent"] = np.nan
  dta["mig_sent_sd"] = np.nan
  dta["mig_sent_n"] = np.nan

  for row in range(dta.shape[0]):

    ## print progress
    print(f"\r Row {row} of {dta.shape[0]}", end="")

    ## split sentences from cleaned text
    sentences = sent_tokenize(dta.loc[row, "text"].replace('„|”|"', " ").replace('!', "! "))

    ## filter sentences about migration    
    rel_sents = []
    for sent in sentences:
      ### tokenize and lowercase tokens of the sentence
      tokenized_sent = [word.lower() for word in word_tokenize(sent)]
      ### if any item in the tokenized sentence is a keyword, append the original sentence
      if any(keyw in tokenized_sent for keyw in mig_dict):
          rel_sents.append(sent)

    if len(rel_sents)>0:
      ## estimate migration sentiment
      results = classifier(rel_sents)
      estimates = []
      for result in results:
        if result["label"] == "positive":
          estimates.append(result["score"])
        elif result["label"] == "neutral":
          estimates.append(0)
        elif result["label"] == "negative":
          estimates.append(result["score"]*-1)
    else:
      estimates = [np.nan]

    ## calculate mean, sd & n of sentences
    dta.loc[row, "mig_sent"] = np.mean(estimates)
    dta.loc[row, "mig_sent_sd"] = np.std(estimates)
    dta.loc[row, "mig_sent_n"] = len(estimates)

  # write to csv
  dta.to_csv("".join(["drive/MyDrive/Bild/", paper, "_estimates.csv"]))

## Limitations
This approach has the problem that the sentiment might not be directed at migration, but in fact talk about the horrible conditions of migration, or something else. Hence, a second approach might replace all migration-related terms with a single token, and track their correlation with specific terms or frames across time.